In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
import tensorflow as tf

tensorflow は  
・グラフの作成 (composed from single node and its relations like theoretical)  
・作成したグラフをセッションで実行  
の二つのフェーズが必要

グラフをセッションで実行するためには、
グラフ内の変数を初期化する必要がある。

実行はノードで管理されており、欲しい値を持つノードを実行する感じ。

### グラフの管理

In [2]:
# 作成したノードは自動的にデフォルトグラフに追加される
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [3]:
tf.reset_default_graph()

In [5]:
# withブロックで一時的なデフォルトグラフの指定ができる
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    print("In default block : x2 is default graph? - ", x2.graph is tf.get_default_graph())
print("Out default block : x2 is default graph? - ", x2.graph is tf.get_default_graph())

In default block : x2 is default graph? -  True
Out default block : x2 is default graph? -  False


### セッションの管理

In [7]:
x1 = tf.Variable(1)

sess = tf.Session()

# 変数の初期化
# 初期化ノードを実行する
sess.run(x1.initializer)
# x.initializer.run()

# グラフの実行
x_res = sess.run(x1)
# x_res = x1.eval()

sess.close()

print(x_res)

1


In [8]:
### その他の方法
x1 = tf.Variable(1)
init = tf.global_variables_initializer()

# # セッションはwithブロックでもできる
# with tf.Session() as sess:
#     sess.run(init)
# #     init.run()
#     x_res = x1.eval()

# シェルなどのインタラクティブコンソールでは InteractiveSession を使用した方が良い
sess = tf.InteractiveSession()
init.run()
x_res = x1.eval()
sess.close()
print(x_res)

1


### その他の変数の初期化について

In [ ]:
# 1 all_variables are deprecated
tf.initialize_all_variables() # == tf.initialize_variables(tf.all_variables())
tf.initialize_local_variables() # == tf.initialize_variables(tf.local_variables())
"""
initializes variables in GraphKeys.VARIABLES and GraphKeys.LOCAL_VARIABLE collections, respectively.
Variables in GraphKeys.LOCAL_VARIABLES collection are variables that are added to the graph, 
but not saved or restored.

tf.Variable() by default adds a new variable to GraphKeys.VARIABLE collection,
which can be controlled by collections=[tf.GraphKeys.LOCAL_VARIABLES/VARIABLES] argument.

LOCAL_VARIABLES: the subset of Variable objects that are local to each machine.
Usually used for temporarily variables, like counters. 

Note: use tf.contrib.framework.local_variable to add to this collection.
"""

# 2 deprecated?
init = tf.group(tf.initialize_all_variables(),
                   tf.initialize_local_variables())

# 3
init_global = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()

### ノードの値のライフサイクル

In [ ]:
w = tf.constant(3)
x = w + 2
y = y + 5
z = x * 3

with tf.Session() as sess:
    # この方法だと、wとxの計算が2回行われる
    # グラフを実行するたびに、ノードの値は捨てられるため。
#     print(y.eval())
#     print(z.eval())
    
    # 次のようにすると、wとxの計算は一回だけになる
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

### 線形回帰

In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape
print(m, n)
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
print(housing_data_plus_bias.shape)

20640 8
(20640, 9)


In [3]:
data = pd.DataFrame(housing.data)

In [4]:
data.head()

,0,1,2,3,4,5,6,7
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [5]:
data.isnull().all()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
dtype: bool

### 正規方程式

In [6]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [7]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [8]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

### マニュアル勾配計算

In [9]:
tf.reset_default_graph()

In [6]:
n_epochs = 1000
learning_rate = 1e-7

In [11]:
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [12]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch = ", epoch, " : MSE = ", mse.eval())
        sess.run(training_op)

Epoch =  0  : MSE =  1336150.1
Epoch =  100  : MSE =  nan
Epoch =  200  : MSE =  nan
Epoch =  300  : MSE =  nan
Epoch =  400  : MSE =  nan
Epoch =  500  : MSE =  nan
Epoch =  600  : MSE =  nan
Epoch =  700  : MSE =  nan
Epoch =  800  : MSE =  nan
Epoch =  900  : MSE =  nan


### 自動微分

In [ ]:
tf.reset_default_graph()

In [13]:
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [14]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    grad = gradients.eval()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch = ", epoch, " : MSE = ", mse.eval())
        sess.run(training_op)

Epoch =  0  : MSE =  2192306.8
Epoch =  100  : MSE =  nan
Epoch =  200  : MSE =  nan
Epoch =  300  : MSE =  nan
Epoch =  400  : MSE =  nan
Epoch =  500  : MSE =  nan
Epoch =  600  : MSE =  nan
Epoch =  700  : MSE =  nan
Epoch =  800  : MSE =  nan
Epoch =  900  : MSE =  nan


### オプティマイザを使用

In [15]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

In [16]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    grad = gradients.eval()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch = ", epoch, " : MSE = ", mse.eval())
        sess.run(training_op)

Epoch =  0  : MSE =  983533.94
Epoch =  100  : MSE =  nan
Epoch =  200  : MSE =  nan
Epoch =  300  : MSE =  nan
Epoch =  400  : MSE =  nan
Epoch =  500  : MSE =  nan
Epoch =  600  : MSE =  nan
Epoch =  700  : MSE =  nan
Epoch =  800  : MSE =  nan
Epoch =  900  : MSE =  nan


### プレースホルダーとミニバッチ学習 (ついでにモデル・変数の保存とグラフ・訓練曲線の可視化)

In [4]:
tf.reset_default_graph()

In [5]:
n_epochs = 1000
learning_rate = 5e-8
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

In [6]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="prediction")

# 名前スコープ
with tf.name_scope("loss") as scope:
#     error = y_pred - y
#     mse = tf.reduce_mean(tf.square(error), name="mse")
    mse = tf.reduce_mean(tf.losses.mean_squared_error(y, y_pred), name="mse")
    
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [6]:
def fetch_batch(epoch, batch_index, batch_size):
    if batch_size*(batch_index+1) < len(housing_data_plus_bias):
        X_batch = housing_data_plus_bias[batch_size*batch_index:batch_size*(batch_index+1)].astype(np.float32)
        y_batch = housing.target.reshape(-1,1)[batch_size*batch_index:batch_size*(batch_index+1)].astype(np.float32)
    else:
        X_batch = housing_data_plus_bias[batch_size*batch_index:].astype(np.float32)
        y_batch = housing.target.reshape(-1,1)[batch_size*batch_index:].astype(np.float32)
    return X_batch, y_batch

In [7]:
X_batch, y_batch = fetch_batch(0, 206, batch_size)
print(X_batch.shape, y_batch.shape)
print(X_batch.dtype, y_batch.dtype)

(40, 9) (40, 1)
float32 float32


In [8]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
# saver = tf.train.Saver({"weights": theta})
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
#     grad = gradients.eval()
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
        
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
                
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        
        if epoch % 100 == 0:
            save_path = saver.save(sess, "./logs/test_model_epoch_{}.ckpt".format(epoch))
            print("Epoch = ", epoch, " : MSE = ", mse.eval(feed_dict={X: X_batch, y: y_batch}))
            
    save_path = saver.save(sess, "./logs/test_model_epoch_{}.ckpt".format(n_epochs))
    
file_writer.close()

Epoch =  0  : MSE =  112.85476
Epoch =  100  : MSE =  43.320763
Epoch =  200  : MSE =  26.188023
Epoch =  300  : MSE =  15.929834
Epoch =  400  : MSE =  9.785694
Epoch =  500  : MSE =  6.1036954
Epoch =  600  : MSE =  3.8952625
Epoch =  700  : MSE =  2.5688667
Epoch =  800  : MSE =  1.7705301
Epoch =  900  : MSE =  1.2885033


In [13]:
# !tensorboard --logdir tf_logs/

### 保存したモデルや変数の取り出し

In [27]:
saver = tf.train.import_meta_graph("./logs/test_model_epoch_1000.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, "./logs/test_model_epoch_1000.ckpt")

INFO:tensorflow:Restoring parameters from ./logs/test_model_epoch_1000.ckpt


### モジュール性と変数の共有

In [ ]:
tf.reset_default_graph()

In [19]:
# def relu(X):
#     w_shape = (int(X.get_shape()[1]), 1)
#     w = tf.Variable(tf.random_normal(w_shape), name="weights")
#     b = tf.Variable(0.0, name="bias")
#     z = tf.add(tf.matmul(X, w), b, name="z")
#     return tf.maximum(z, 0., name="relu")

In [20]:
# n_features = 3
# X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
# # 変数は共有されない
# relus = [relu(X) for i in range(5)]
# output = tf.add_n(relus, name="output")

In [21]:
# 名前スコープによるモジュール化
def relu_share_w(X, w):
    with tf.name_scope("relu"):
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")

In [23]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
# 共有したい変数はconfigなどに設定するといい感じらしい
w_shape = (int(X.get_shape()[1]), 1)
w = tf.Variable(tf.random_normal(w_shape), name="weights")
# 変数wが共有される
relus = [relu_share_w(X, w) for i in range(5)]
output = tf.add_n(relus, name="output")

In [26]:
hasattr(relu, "w_shape")

False

In [27]:
def aaa(a):
    aaa.abc = 1
    return a * aaa.abc

In [28]:
aaa(3)

3

In [30]:
hasattr(aaa, "abc")

True

In [ ]:
tf.reset_default_graph()

In [ ]:
# # 共有したい変数を関数の属性に追加してしまおう的な
# def relu_with_attr(X):
#     with tf.name_scope("relu"):
#         if not hasattr(relu_with_attr, "w"):
#             w_shape = (int(X.get_shape()[1]), 1)
#             relu_with_attr.w = tf.Variable(tf.random_normal(w_shape), name="weights")
#         b = tf.Variable(0.0, name="bias")
#         z = tf.add(tf.matmul(X, w), b, name="z")
#         return tf.maximum(z, 0., name="max")
    
# # RELUクラスを作ってしまえば、変数の共有も楽チン (少し重い)
# class RELU():
#     __init__(self, X):
#         w_shape = (int(X.get_shape()[1]), 1)
#         self.w = tf.Variable(tf.random_normal(w_shape), name="weigths")
#     ### ....的な感じで

In [ ]:
# tf.reset_default_graph()

In [10]:
# 既存の変数があればそちらを使い、なければ新しく変数を作る
# # reuse=Trueを指定しないと変数があった場合に例外が発生する
# with tf.variable_scope("relu"):
    # イニシャライザーがスカラの時はshape=()とする
#     threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    
# reuse=Trueを指定すると、変数が再利用される (形状や初期化の指定をする必要はない。変数がすでにある場合は例外となる。)
# with tf.variable_scope("relu", reuse=True) as scope:
#     # スコープ内でも、スコープに対してreuseを設定できる
# #     scope.reuse_variables()
#     w_shape = (int(X.get_shape()[1]), 1)
#     # 指定するイニシャライザーがイニシャライズインスタンスではなく定数であるなら、shapeの指定はしない
#     w = tf.get_variable("weights", initializer=tf.random_normal(w_shape))
#     # reuse=Trueのスコープ内で新たにスコープを作成しても、reuse=Trueのままになる。(スコープの中でFalseにできない)
# #     with tf.variable_scope("threshold"):
# #         threshold = tf.get_variable("threshold_value")

# # reuse=tf.AUTO_REUSEとすると、例外を発生させることなく、既存の変数があればそちらを使い、なければ新しく変数を作る
# with tf.variable_scope("relu", reuse=tf.AUTO_REUSE) as scope:
#     w_shape = (int(X.get_shape()[1]), 1)
#     w = tf.get_variable("weights", initializer=tf.random_normal(w_shape))

In [12]:
### まとめると、、、
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
n_features = n + 1

tf.reset_default_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

# X_input = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X_input")

def relu(X):
    with tf.variable_scope("relu", reuse=tf.AUTO_REUSE):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.get_variable("weights", initializer=tf.random_normal(w_shape))
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="max")

with tf.Graph().as_default() as graph:
    X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
    relus = []
    for relu_index in range(5):
        relus.append(relu(X))
    output = tf.reduce_mean(tf.add_n(relus, name="output"))

    init = tf.global_variables_initializer()
    output_summary = tf.summary.scalar('res', output)
    with tf.summary.FileWriter(log_dir, graph) as file_writer:
    
        with tf.Session() as sess:
            sess.run(init)
            summary_str = output_summary.eval(feed_dict={X: housing_data_plus_bias})
            file_writer.add_summary(summary_str, 0)

20640 8
